In [ ]:
!pip install -U sentence-transformers
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 4.2 MB 10.4 MB/s 
     |████████████████████████████████| 1.2 MB 57.0 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 67.7 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=9a4de93733cf1ae6a73196b85ab7322292a563eb8b53c09deb556d2a784de5a2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [ ]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.2 MB/s 
     |████████████████████████████████| 581 kB 63.0 MB/s 
     |████████████████████████████████| 235 kB 70.7 MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import torch

model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from sentence_transformers import util
from underthesea import sent_tokenize
from underthesea import word_tokenize

In [ ]:
def find_top_k(top_k, model, question, corpus, corpus_embeddings):
  k = min(top_k, len(corpus))
  query_embedding = model.encode(question)

  cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
  # print("error")
  # cos_scores = cos_scores.cpu()

  top_results = torch.topk(cos_scores, k=k)

  return top_results[1]

In [ ]:
from copy import deepcopy
def update_top_10(train_data, k):
  copied_data = deepcopy(train_data)
  for topic in copied_data['data']:
    for paragraph in topic['paragraphs']:
      # paragraph['context'] = sent_tokenize(paragraph['context'])
      context = paragraph['context']
      context_embeddings = model.encode(context)
      for qa in paragraph['qas']:
        question = qa['question']

        ## Get top k most relevant sentences
        top_k_candidates = find_top_k(k, model, question, context, context_embeddings)
        qa['candidate_indices'] = top_k_candidates.tolist()
  return copied_data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/vdt_dsai/dataset"

/content/drive/MyDrive/vdt_dsai/dataset


In [ ]:
!ls

test_file.json	train_file.json  val_file.json


In [ ]:
import json
with open("test_file.json") as f:
  data =  json.load(f)

In [ ]:
new_data_train = update_top_10(data, 10)

In [ ]:
def find_sentence(corpus, start_answer):
    '''
    Find sentence contains answer
    * corpus: list sentences of context
    * start_answer: the begin position of answer in context
    '''
    start, end = 0, -1
    sent_idx = -1
    for i in range(len(corpus)):
        sentence = corpus[i]
        start = end + 1
        end = start+len(sentence)
        if start<=start_answer<end:
            sent_idx = i
            break
        elif start_answer == end:
            sent_idx = i + 1
            break 
    return sent_idx

def accuracy(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    invalid = 0 # number of invalid qa
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:

                    # If the answer is from multiple sentences then discard this qa
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
                        invalid += 1
                        continue 

                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))

                    # Find the index of sentence that contains the answer 
                    ans_sent_idx = find_sentence(context, start_answer)
                    # Check if the above sentence is retrived in top_k 
                    results.append(ans_sent_idx in qa['candidate_indices'][:k])
    return {"top_k":top_k, "# valid qa":len(results), "# invalid qa": invalid,  "true":sum(results), "accuracy":round(sum(results) / len(results), 3)}

In [ ]:
print("SimCSE Test")
simcse_test_results = []
for i in range(10):
    json_i = accuracy(new_data_train, i+1)
    simcse_test_results.append(json_i)
import pandas as pd 
simcse_test_df = pd.read_json(json.dumps(simcse_test_results))
simcse_test_df

SimCSE Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1156,0.680
1,2,1701,3,1444,0.849
2,3,1701,3,1578,0.928
3,4,1701,3,1639,0.964
4,5,1701,3,1667,0.980
5,6,1701,3,1682,0.989
6,7,1701,3,1693,0.995
7,8,1701,3,1696,0.997
8,9,1701,3,1699,0.999
9,10,1701,3,1701,1.000


In [ ]:
def inspect(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1: break 
                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))
                    ans_sent_idx = find_sentence(context, start_answer)
                    flag = ans_sent_idx in qa['candidate_indices'][:k]
                    results.append(flag)
                    if (flag == False):
                      print(qa['question'])
                      print(qa['answers'][0]['text'])
                      print("True sentence:", context[ans_sent_idx])
                      print(context)
                      print(ans_sent_idx, start_answer)
                      print("Best candidates: \n" + context[qa['candidate_indices'][0]])
                      print("")

    # return {"top_k":top_k, "total":len(results), "true":sum(results), "accuracy":round(sum(results) / len(results), 3)}

inspect(new_data_train, 10)

In [ ]:
corpus = ['Việc rút đi của hơn nửa triệu quân Mỹ đã để lại một khoảng trống khổng lồ trong nền kinh tế: hàng tỷ đôla hàng năm trước đây được lính viễn chinh Mỹ tung vào xã hội qua các dịch vụ mua sắm ở thành phố, nay không còn nữa.', 'Một khối lượng lớn người lao động làm việc trong các “sở Mỹ” (các văn phòng, trụ sở quân sự của Mỹ) cũng không còn việc làm.', 'Số lượng công nhân làm ở các cơ sở của Mỹ năm 1971 là 100.000 người, đến tháng 12 năm 1972 chỉ còn lại 10.000 người, tạo ra thất nghiệp hàng loạt".', 'Theo phúc trình của VECCO xuất bản tháng 1/1975 thì: Sài Gòn năm 1974 có 3 triệu dân thì có đến 600.000 người thất nghiệp.', 'Chênh lệch giàu nghèo rất lớn khi thu nhập của thiểu số "tầng lớp trên" chiếm 43,5% GDP, tầng lớp dưới chỉ đạt 1,8%.']
start_answer = 220
idx = find_sentence(corpus, start_answer)
corpus[idx]

'Một khối lượng lớn người lao động làm việc trong các “sở Mỹ” (các văn phòng, trụ sở quân sự của Mỹ) cũng không còn việc làm.'